# BQML - Linear Regression

In [1]:
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
PROJECT_NUM = !gcloud projects list --filter="$PROJECT_ID" --format="value(PROJECT_NUMBER)"
PROJECT_NUM = PROJECT_NUM[0]
LOCATION = 'us-central1'
REGION = "us-central1"

# VERTEX_SA = '934903580331-compute@developer.gserviceaccount.com'
VERTEX_SA = 'jt-vertex-sa@hybrid-vertex.iam.gserviceaccount.com'

print(f"PROJECT_ID: {PROJECT_ID}")
print(f"PROJECT_NUM: {PROJECT_NUM}")
print(f"LOCATION: {LOCATION}")
print(f"REGION: {REGION}")
print(f"VERTEX_SA: {VERTEX_SA}")

PROJECT_ID: hybrid-vertex
PROJECT_NUM: 934903580331
LOCATION: us-central1
REGION: us-central1
VERTEX_SA: jt-vertex-sa@hybrid-vertex.iam.gserviceaccount.com


In [2]:
VERSION='av3'

In [3]:
REGION = 'us-central1'
EXPERIMENT = 'forecasting-1'
SERIES = f'{VERSION}-forecasting'

BQ_PROJECT = PROJECT_ID
BQ_DATASET = SERIES.replace('-','_')
BQ_TABLE = 'forecasting-1'

viz_limit = 12

VERTEX_AI_MODEL_ID='v2_mlr'
MODEL_VERSION='v7'
XAI_FLAG="TRUE"

# CUSTOMIZE
FORECAST_GRANULARITY = 'DAILY' # the data preparation included preparing the data at this level
FORECAST_HORIZON_LENGTH = 14 #2 # 7# 14
FORECAST_TEST_LENGTH = 14 # the data preparation included setting this value for splits = TEST
FORECAST_VALIDATE_LENGTH = 14 # the data preparation included setting this value for splits = VALIDATE

EXPERIMENT_NAME = f"nyc_{BQ_DATASET}"
print(f'EXPERIMENT_NAME: {EXPERIMENT_NAME}')

EXPERIMENT_NAME: nyc_av3_forecasting


In [4]:
from google.cloud import bigquery

import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, timedelta

from google.cloud import aiplatform as vertex_ai

In [5]:
bq = bigquery.Client(project=PROJECT_ID)

vertex_ai.init(
    project=PROJECT_ID, 
    location=REGION,
    # credentials=credentials
)

## review time series data

In [6]:
# CUSTOMIZE
TARGET_COLUMN = 'num_trips'
TIME_COLUMN = 'starttime'
SERIES_COLUMN = 'start_station_name'
SPLIT_COLUMN = 'splits'
COVARIATE_COLUMNS = ['avg_tripduration', 'pct_subscriber', 'ratio_gender', 'capacity'] # could be empty

# # CUSTOMIZE
# FORECAST_GRANULARITY = 'DAILY' # the data preparation included preparing the data at this level
# FORECAST_HORIZON_LENGTH = 14 #2 # 7# 14
# FORECAST_TEST_LENGTH = 14 # the data preparation included setting this value for splits = TEST
# FORECAST_VALIDATE_LENGTH = 14 # the data preparation included setting this value for splits = VALIDATE

In [7]:
query = f"""
    WITH
        SPLIT AS (
            SELECT {SPLIT_COLUMN}, min({TIME_COLUMN}) as mindate, max({TIME_COLUMN}) as maxdate
            FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped`
            GROUP BY {SPLIT_COLUMN}
        ),
        TRAIN AS (
            SELECT mindate as start_date
            FROM SPLIT
            WHERE {SPLIT_COLUMN} ='TRAIN'
        ),
        VAL AS (
            SELECT mindate as val_start
            FROM SPLIT
            WHERE {SPLIT_COLUMN} = 'VALIDATE'
        ),
        TEST AS (
            SELECT mindate as test_start, maxdate as end_date
            FROM SPLIT
            WHERE {SPLIT_COLUMN} = 'TEST'
        )
    SELECT * EXCEPT(pos) FROM
    (SELECT *, ROW_NUMBER() OVER() pos FROM TRAIN)
    JOIN (SELECT *, ROW_NUMBER() OVER() pos FROM VAL)
    USING (pos)
    JOIN (SELECT *, ROW_NUMBER() OVER() pos FROM TEST)
    USING (pos)
"""
keyDates = bq.query(query).to_dataframe()
keyDates

,start_date,val_start,test_start,end_date
0,2013-07-01,2016-09-03,2016-09-17,2016-09-30


In [8]:
query = f"""
    SELECT {SERIES_COLUMN}, {TIME_COLUMN}, {TARGET_COLUMN}, {SPLIT_COLUMN},
        {', '.join(COVARIATE_COLUMNS)}
    FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped`
    ORDER by {SERIES_COLUMN}, {TIME_COLUMN}
"""
rawSeries = bq.query(query).to_dataframe()

In [9]:
EX1_CTE = f"""
WITH EX1_CTE AS (
    SELECT
        {TARGET_COLUMN},
        {SERIES_COLUMN},
        {SPLIT_COLUMN},
        {', '.join(COVARIATE_COLUMNS)},
        
        # TIME FEATURES
        EXTRACT(YEAR FROM {TIME_COLUMN}) AS Year,
        EXTRACT(MONTH FROM {TIME_COLUMN}) AS Month,
        EXTRACT(DAY FROM {TIME_COLUMN}) AS Day_of_Month,
        EXTRACT(DAYOFYEAR FROM {TIME_COLUMN}) AS Day_of_Year,
        EXTRACT(DAYOFWEEK FROM {TIME_COLUMN}) AS Day_of_Week,
        CASE WHEN EXTRACT(DAYOFWEEK FROM {TIME_COLUMN}) IN (1, 7) THEN 1 ELSE 0 END AS Weekend,
        
    FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped`
)
"""

In [10]:
BQ_MODEL = f"{BQ_TABLE}_mlr_{MODEL_VERSION}"
query = f"""
CREATE OR REPLACE MODEL `{BQ_PROJECT}.{BQ_DATASET}.{BQ_MODEL}`
    OPTIONS (
        model_type = 'LINEAR_REG',
        input_label_cols = ['{TARGET_COLUMN}'],
        data_split_col = 'custom_splits',
        data_split_method = 'CUSTOM',
        MAX_ITERATIONS=3,
       -- Vertex AI fields
       model_registry="vertex_ai", 
       vertex_ai_model_id='{VERTEX_AI_MODEL_ID}',
       vertex_ai_model_version_aliases=['{MODEL_VERSION}', 'experimental']
       -- enable_global_explain={XAI_FLAG}
    ) AS
    {EX1_CTE}
    SELECT
        * EXCEPT({SPLIT_COLUMN}),
        CASE
            WHEN {SPLIT_COLUMN} = 'TRAIN' THEN FALSE
            ELSE TRUE
        END AS custom_splits
    FROM EX1_CTE
    WHERE {SPLIT_COLUMN} in ('TRAIN', 'VALIDATE')
"""
print(query)


CREATE OR REPLACE MODEL `hybrid-vertex.av3_forecasting.forecasting-1_mlr_v7`
    OPTIONS (
        model_type = 'LINEAR_REG',
        input_label_cols = ['num_trips'],
        data_split_col = 'custom_splits',
        data_split_method = 'CUSTOM',
        MAX_ITERATIONS=3,
       -- Vertex AI fields
       model_registry="vertex_ai", 
       vertex_ai_model_id='v2_mlr',
       vertex_ai_model_version_aliases=['v7', 'experimental']
       -- enable_global_explain=TRUE
    ) AS
    
WITH EX1_CTE AS (
    SELECT
        num_trips,
        start_station_name,
        splits,
        avg_tripduration, pct_subscriber, ratio_gender, capacity,
        
        # TIME FEATURES
        EXTRACT(YEAR FROM starttime) AS Year,
        EXTRACT(MONTH FROM starttime) AS Month,
        EXTRACT(DAY FROM starttime) AS Day_of_Month,
        EXTRACT(DAYOFYEAR FROM starttime) AS Day_of_Year,
        EXTRACT(DAYOFWEEK FROM starttime) AS Day_of_Week,
        CASE WHEN EXTRACT(DAYOFWEEK FROM starttime) IN (1, 

In [11]:
job = bq.query(query = query)
job.result()
print(job.state, (job.ended-job.started).total_seconds())

DONE 14.872


In [12]:
print(f'Direct link to the model in BigQuery:\nhttps://console.cloud.google.com/bigquery?project={PROJECT_ID}&ws=!1m5!1m4!5m3!1s{PROJECT_ID}!2s{BQ_DATASET}!3s{BQ_MODEL}')


Direct link to the model in BigQuery:
https://console.cloud.google.com/bigquery?project=hybrid-vertex&ws=!1m5!1m4!5m3!1shybrid-vertex!2sav3_forecasting!3sforecasting-1_mlr_v7


In [13]:
query = f"""
SELECT *
FROM ML.WEIGHTS (MODEL `{BQ_PROJECT}.{BQ_DATASET}.{BQ_MODEL}`)
"""
weights = bq.query(query = query).to_dataframe()
weights

,processed_input,weight,category_weights
0,start_station_name,NaN,"[{'category': 'Central Park West & W 102 St', ..."
1,avg_tripduration,-0.000582,[]
2,pct_subscriber,-213.188933,[]
3,ratio_gender,-1.091443,[]
4,capacity,-0.658549,[]
5,Year,24.909874,[]
6,Month,310.285385,[]
7,Day_of_Month,10.161219,[]
8,Day_of_Year,-10.020749,[]
9,Day_of_Week,-0.609004,[]


In [15]:
weights['category_weights'].iloc[0]

array([{'category': 'Central Park West & W 102 St', 'weight': 315.2377054592557},
       {'category': 'Central Park West & W 76 St', 'weight': 300.8292194313999},
       {'category': 'Central Park W & W 96 St', 'weight': 351.28727279169794},
       {'category': 'Central Park West & W 100 St', 'weight': 284.63304194353293},
       {'category': 'W 106 St & Central Park West', 'weight': 323.1312168209622},
       {'category': 'Grand Army Plaza & Central Park S', 'weight': 465.8462236617953},
       {'category': 'Central Park North & Adam Clayton Powell Blvd', 'weight': 254.198932233677},
       {'category': 'Central Park S & 6 Ave', 'weight': 418.52706415860723},
       {'category': 'W 82 St & Central Park West', 'weight': 341.4598866184814},
       {'category': 'Central Park West & W 72 St', 'weight': 376.78011332659105},
       {'category': 'Central Park West & W 68 St', 'weight': 366.6210688866532},
       {'category': 'Central Park West & W 85 St', 'weight': 385.8770427198153}],
     

In [16]:
query = f"""
{EX1_CTE}
SELECT 'TEST' as SPLIT, * FROM ML.EVALUATE (MODEL `{BQ_PROJECT}.{BQ_DATASET}.{BQ_MODEL}`,
    (SELECT * FROM EX1_CTE WHERE {SPLIT_COLUMN} = 'TEST'))
UNION ALL
SELECT 'VALIDATE' as SPLIT, * FROM ML.EVALUATE (MODEL `{BQ_PROJECT}.{BQ_DATASET}.{BQ_MODEL}`,
    (SELECT * FROM EX1_CTE WHERE {SPLIT_COLUMN} = 'VALIDATE'))
UNION ALL
SELECT 'TRAIN' as SPLIT, * FROM ML.EVALUATE (MODEL `{BQ_PROJECT}.{BQ_DATASET}.{BQ_MODEL}`,
    (SELECT * FROM EX1_CTE WHERE {SPLIT_COLUMN} = 'TRAIN'))
"""
bq.query(query = query).to_dataframe()

,SPLIT,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,TEST,57.448783,6548.842378,0.726357,39.449742,0.357435,0.622654
1,TRAIN,45.374309,3732.591382,0.537276,34.489672,0.504250,0.504250
2,VALIDATE,59.484503,6872.123524,0.479323,40.664097,0.393723,0.626341


### Review Custom Metrics with SQL

Some common metrics for evaluating forecasting effectiveness are 
- MAPE, or Mean Absolute Percentage Error
    - $\textrm{MAPE} = \frac{1}{n}\sum{\frac{\mid(actual - forecast)\mid}{actual}}$
- MAE, or Mean Absolute Error
     - $\textrm{MAE} = \frac{1}{n}\sum{\mid(actual - forecast)\mid}$
- MAE divided by average demand so it yields a % like MAPE
    - $\textrm{pMAE} = \frac{\sum{\mid(actual - forecast)\mid}}{\sum{actual}}$
- MSE, or Mean Squared Error
    - $\textrm{MSE} = \frac{1}{n}\sum{(actual-forecast)^2}$
- RMSE, or Root Mean Squared Error
    - $\textrm{RMSE} = \sqrt{\frac{1}{n}\sum{(actual-forecast)^2}}$
- RMSE divided by average demand so it yeilds a % like MAPE
    - $\textrm{pRMSE} = \frac{\sqrt{\frac{1}{n}\sum{(actual-forecast)^2}}}{\frac{1}{n}\sum{actual}}$

It can be helpful to explicity caculate these to make comparison between datasets and models fair.  This section demonstration these calculation with SQL.

In [17]:
query = f"""
{EX1_CTE},
        DIFFS AS (
            SELECT {SERIES_COLUMN},  
                {TARGET_COLUMN} as actual_value,
                predicted_{TARGET_COLUMN} as forecast_value, 
                ({TARGET_COLUMN} - predicted_{TARGET_COLUMN}) as diff
            FROM ML.PREDICT (MODEL `{BQ_PROJECT}.{BQ_DATASET}.{BQ_MODEL}`, (
                SELECT *
                FROM EX1_CTE
                WHERE {SPLIT_COLUMN} = 'TEST'
            ))
        )
    SELECT {SERIES_COLUMN}, 
            AVG(SAFE_DIVIDE(ABS(diff), actual_value)) as MAPE,
            AVG(ABS(diff)) as MAE,
            SAFE_DIVIDE(SUM(ABS(diff)), SUM(actual_value)) as pMAE,
            AVG(POW(diff, 2)) as MSE,
            SQRT(AVG(POW(diff, 2))) as RMSE,
            SAFE_DIVIDE(SQRT(AVG(POW(diff, 2))), AVG(actual_value)) as pRMSE
    FROM DIFFS
    GROUP BY {SERIES_COLUMN}
    ORDER BY {SERIES_COLUMN}    
"""
customMetrics_ex1 = bq.query(query = query).to_dataframe()
customMetrics_ex1

,start_station_name,MAPE,MAE,pMAE,MSE,RMSE,pRMSE
0,Central Park North & Adam Clayton Powell Blvd,0.937319,127.340635,0.820418,19155.889904,138.404804,0.891701
1,Central Park S & 6 Ave,0.485674,158.592184,0.485947,34053.363213,184.535534,0.565440
2,Central Park W & W 96 St,0.396896,35.582368,0.355570,1644.105371,40.547569,0.405186
3,Central Park West & W 100 St,0.890347,28.196119,0.711253,1184.708864,34.419600,0.868242
4,Central Park West & W 102 St,0.762395,32.787437,0.647425,1392.820513,37.320511,0.736935
5,Central Park West & W 68 St,0.344680,57.045808,0.376540,4625.396357,68.010267,0.448913
6,Central Park West & W 72 St,0.299342,54.938694,0.310523,4181.733784,64.666327,0.365505
7,Central Park West & W 76 St,0.354958,35.710059,0.326119,1548.448125,39.350326,0.359364
8,Central Park West & W 85 St,0.416622,31.290689,0.248903,1481.648610,38.492189,0.306188
9,Grand Army Plaza & Central Park S,0.354344,51.661987,0.237568,3667.488515,60.559793,0.278485


In [18]:
query = f"""
{EX1_CTE},
        DIFFS AS (
            SELECT {SERIES_COLUMN},  
                {TARGET_COLUMN} as actual_value,
                predicted_{TARGET_COLUMN} as forecast_value, 
                ({TARGET_COLUMN} - predicted_{TARGET_COLUMN}) as diff
            FROM ML.PREDICT (MODEL `{BQ_PROJECT}.{BQ_DATASET}.{BQ_MODEL}`, (
                SELECT *
                FROM EX1_CTE
                WHERE {SPLIT_COLUMN} = 'TEST'
            ))
        )
    SELECT #{SERIES_COLUMN}, 
            AVG(SAFE_DIVIDE(ABS(diff), actual_value)) as MAPE,
            AVG(ABS(diff)) as MAE,
            SAFE_DIVIDE(SUM(ABS(diff)), SUM(actual_value)) as pMAE,
            AVG(POW(diff, 2)) as MSE,
            SQRT(AVG(POW(diff, 2))) as RMSE,
            SAFE_DIVIDE(SQRT(AVG(POW(diff, 2))), AVG(actual_value)) as pRMSE
    FROM DIFFS
    #GROUP BY {SERIES_COLUMN}
    #ORDER BY {SERIES_COLUMN}    
"""
customMetrics_ex1 = bq.query(query = query).to_dataframe()
customMetrics_ex1

,MAPE,MAE,pMAE,MSE,RMSE,pRMSE
0,0.513595,57.448783,0.428944,6548.842378,80.924918,0.604229


## Log Vertex Experiments

In [19]:
from datetime import datetime

# create run name
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
EXPERIMENT_RUN_NAME = f"run-{TIMESTAMP}"

# log params and metrics to dicts
params = {}
# params["budget_hrs"] = MILLI_NODE_HRS
params["horizon"] = FORECAST_HORIZON_LENGTH
# params["context_window"] = CONTEXT_WINDOW
params["model_type"] = "mlr"

metrics_dict = {}
metrics_dict["MAPE"] = customMetrics_ex1['MAPE'][0]
metrics_dict["MAE"] = customMetrics_ex1['MAE'][0]
metrics_dict["pMAE"] = customMetrics_ex1['pMAE'][0]
metrics_dict["MSE"] = customMetrics_ex1['MSE'][0]
metrics_dict["RMSE"] = customMetrics_ex1['RMSE'][0]
metrics_dict["pRMSE"] = customMetrics_ex1['pRMSE'][0]

# # Create and log experiment
vertex_ai.init(experiment=EXPERIMENT_NAME.replace("_","-"))

with vertex_ai.start_run(EXPERIMENT_RUN_NAME) as my_run:
    my_run.log_metrics(metrics_dict)
    my_run.log_params(params)

    vertex_ai.end_run()

Associating projects/934903580331/locations/us-central1/metadataStores/default/contexts/nyc-av3-forecasting-run-20230515142138 to Experiment: nyc-av3-forecasting


## Forecast TEST

In [20]:
query = f"""
{EX1_CTE}
SELECT
    {SERIES_COLUMN},
    DATE(year, month, day_of_month) AS {TIME_COLUMN},
    predicted_{TARGET_COLUMN} AS predicted
FROM ML.PREDICT (MODEL `{BQ_PROJECT}.{BQ_DATASET}.{BQ_MODEL}`,(
    SELECT *
    FROM EX1_CTE
    )
  )
ORDER BY Year, Month, Day_of_Month
"""
pred_ex1 = bq.query(query = query).to_dataframe()

In [21]:
pred_ex1

,start_station_name,starttime,predicted
0,Central Park S & 6 Ave,2013-07-01,78.019726
1,Grand Army Plaza & Central Park S,2013-07-01,123.266406
2,Grand Army Plaza & Central Park S,2013-07-02,140.436463
3,Central Park S & 6 Ave,2013-07-02,147.744170
4,Grand Army Plaza & Central Park S,2013-07-03,119.038898
...,...,...,...
4377,Central Park West & W 76 St,2016-09-30,7.548854
4378,W 82 St & Central Park West,2016-09-30,10.522474
4379,Grand Army Plaza & Central Park S,2016-09-30,140.905110
4380,Central Park West & W 68 St,2016-09-30,36.545495


# Register model in Vertex AI Model Registry

In [65]:
# Initiate Vertex AI Model Registry for `VERTEX_AI_MODEL_ID` model entry
registry = vertex_ai.models.ModelRegistry(VERTEX_AI_MODEL_ID)

In [66]:
# Get model versions
versions = registry.list_versions()

for version in versions:
    version_id = version.version_id
    version_created_time = datetime.fromtimestamp(
        version.version_create_time.timestamp()
    ).strftime("%m/%d/%Y %H:%M:%S")
    version_aliases = version.version_aliases
    print(
        f"Model version {version_id} was created at {version_created_time} with aliases {version_aliases}",
    )

Getting versions for projects/hybrid-vertex/locations/us-central1/models/v2_mlr
Model version 1 was created at 05/15/2023 11:49:27 with aliases ['default']
Model version 2 was created at 05/15/2023 12:22:43 with aliases []
Model version 3 was created at 05/15/2023 12:30:20 with aliases []
Model version 4 was created at 05/15/2023 13:16:08 with aliases ['v1']
Model version 5 was created at 05/15/2023 13:32:43 with aliases ['v2']
Model version 6 was created at 05/15/2023 13:41:22 with aliases ['v3']
Model version 7 was created at 05/15/2023 13:44:06 with aliases ['v5']
Model version 8 was created at 05/15/2023 13:58:56 with aliases ['v6', 'experimental']
